# Fussing with the QG outputs

In [2]:
# imports
import os

import xarray

import numpy as np
# import fsspec
import matplotlib
import matplotlib.pyplot as plt
import gsw_xarray as gsw
from dask.diagnostics import ProgressBar

# Load up

In [3]:
qg_file = '/home/xavier/Projects/Oceanography/data/QG/QGModelOutput20years.nc'

qg = xarray.open_dataset(qg_file)

# View

In [4]:
qg

<xarray.Dataset> Size: 23GB
Dimensions:  (time: 7200, lev: 2, y: 256, x: 256, lev_mid: 1, l: 256, k: 129)
Coordinates:
  * time     (time) float64 58kB 0.0 8.64e+04 1.728e+05 ... 6.219e+08 6.22e+08
  * lev      (lev) int64 16B 1 2
  * lev_mid  (lev_mid) float64 8B 1.5
  * x        (x) float64 2kB 1.953e+03 5.859e+03 ... 9.941e+05 9.98e+05
  * y        (y) float64 2kB 1.953e+03 5.859e+03 ... 9.941e+05 9.98e+05
  * l        (l) float64 2kB 0.0 6.283e-06 1.257e-05 ... -1.257e-05 -6.283e-06
  * k        (k) float64 1kB 0.0 6.283e-06 1.257e-05 ... 0.000798 0.0008042
Data variables:
    q        (time, lev, y, x) float64 8GB ...
    u        (time, lev, y, x) float64 8GB ...
    v        (time, lev, y, x) float64 8GB ...
Attributes: (12/23)
    pyqg:beta:       1.5e-11
    pyqg:delta:      0.25
    pyqg:del2:       0.8
    pyqg:dt:         3600.0
    pyqg:filterfac:  23.6
    pyqg:L:          1000000.0
    ...              ...
    pyqg:tc:         0
    pyqg:tmax:       622080000.0
    pyqg:twrite:     10000
    pyqg:W:          1000000.0
    title:           pyqg: Python Quasigeostrophic Model
    reference:       https://pyqg.readthedocs.io/en/latest/index.html

----

# Manuel Fig 3

In [ ]:
# Plots histograms SF1
colors = plt.cm.jet(np.linspace(0, 1, 4))

fog1 = plt.figure(figsize=(7, 5.5))

b2 = plt.subplot(2, 2, 1)
b2.semilogx(rr1*1e-3, du1.T*1e3, '-', color='orange', 
            linewidth=0.5, alpha=0.1)
b2.semilogx(0, 0, '-', color='orange', linewidth=0.5, alpha=0.8, label='Daily $\\overline{\\delta u1(r, t)}$')
b2.semilogx(rr1*1e-3, sf1_mn*1e3, 'k', linewidth=1, label='$D1(r)$')
b2.fill_between(rr1*1e-3, (sf1_mn + sf1_std/nu3)*1e3, (sf1_mn - sf1_std/nu3)*1e3, 
               alpha=1, color='black', edgecolor=None)
b2.semilogx(rr1*1e-3, dull_mn*1e3, 'b', linewidth=1, label='$<\\overline{\\delta u_{L}(r, t)}>$')
b2.fill_between(rr1*1e-3, (dull_mn + dull_std/nu3)*1e3, (dull_mn - dull_std/nu3)*1e3, 
               alpha=1, color='blue', edgecolor=None)
b2.semilogx(rr1*1e-3, dutt_mn*1e3, 'r', linewidth=1, label='$<\\overline{\\delta u_{T}(r, t)}$')
b2.fill_between(rr1*1e-3, (dutt_mn + dutt_std/nu3)*1e3, (dutt_mn - dutt_std/nu3)*1e3, 
               alpha=1, color='red', edgecolor=None)
b2.legend(fontsize=8, loc=3, ncol=1)
b2.set_yticks(np.arange(-10, 12, 2))
b2.set_ylim(-6.5, 6.5)
# b2.set_xlabel('$r$ [km]')
b2.set_ylabel('$\\delta u \\times 10^{-3}$ [m s$^{-1}$]')
b2.text(0.1, 0.90, 'a)', bbox=props, transform=b2.transAxes)
b2.set_xticklabels([])
b2.grid()

b4 = plt.subplot(2, 2, 3)
b4.semilogx(rr1*1e-3, sf1_skew, '-k', linewidth=2, label='skewness')
b4.semilogx(rr1*1e-3, sf1_kurt, '--k', linewidth=2, label='kurtosis')
b4.legend(fontsize=8, loc=3)
b4.set_ylim(-1, 1)
b4.set_xlabel('$r$ [km]')
b4.text(0.1, 0.90, 'b)', bbox=props, transform=b4.transAxes)
b4.grid()


b3 = plt.subplot(2, 2, 2)
b3.bar(sf1h0.du1_bin, sf1h0, width=du1r, label="{:2.0f} km".format(rr1[in1-1]*1e-3), 
        color='k', edgecolor=None, alpha=0.4)
b3.plot(d1_bins, sf1p0, '--', color='k', label='Gaussian fit')
b3.bar(sf1h2.du1_bin, sf1h2, width=du1r, label="{:2.0f} km".format(rr1[in2]*1e-3), 
        color='b', edgecolor=None, alpha=0.3)
b3.plot(d2_bins, sf1p2, '--', color='b')
b3.set_xlim(-4.5, 4.5)
b3.set_ylabel('Probability Density')
b3.legend(loc=1, fontsize=8)
b3.text(0.1, 0.90, 'c)', bbox=props, transform=b3.transAxes)
b3.set_xticklabels([])

b1 = plt.subplot(2, 2, 4)
b1.bar(sf1h10.du1_bin, sf1h10, width=du1r, label="{:2.0f} km".format(rr1[in3-1]*1e-3), 
        color='k', edgecolor=None, alpha=0.3)
b1.plot(d3_bins, sf1p10, '--', color='k', label='Gaussian fit')
b1.bar(sf1h15.du1_bin, sf1h15, width=du1r, label="{:2.0f} km".format(rr1[in4]*1e-3), 
        color='b', edgecolor=None, alpha=0.3)
b1.plot(d4_bins, sf1p15, '--', color='b')
b1.set_xlim(-4.5, 4.5)
b1.set_ylabel('Probability Density')
b1.legend(loc=1, fontsize=8)
b1.set_xlabel('$\\delta u/\\sigma_{\\delta u}$')
b1.text(0.1, 0.90, 'd)', bbox=props, transform=b1.transAxes)


fog1.tight_layout(h_pad=-0.1, w_pad=0.13)
plt.savefig('./FiguresMethodsPaper/fig_qg_hist_sf1_N.png', bbox_inches = 'tight', format='png', dpi=500)
